In [158]:
%matplotlib tk
import numpy as np
import hyperspy.api as hs
import pyxem as pxm
import pymatgen as pmg
from matplotlib import pyplot as plt
from transforms3d.euler import euler2axangle
from pymatgen.transformations.standard_transformations import RotationTransformation
from pyxem.generators.indexation_generator import IndexationGenerator
from pyxem.utils.sim_utils import peaks_from_best_template
from pyxem.utils.plot import generate_marker_inputs_from_peaks
from scipy.constants import pi

First we set up two crystal structures

In [159]:
lattice = pmg.Lattice.cubic(5.431)
si = pmg.Structure.from_spacegroup("Fd-3m",lattice, [pmg.Element("Si")], [[0, 0, 0]])

In [160]:
lattice = pmg.Lattice.hexagonal(3,5)
ga = pmg.Structure.from_spacegroup("P6_3mc",lattice, [pmg.Element("Ga")], [[0, 0, 0]])

And some simulation paramaters

In [161]:
size = 256 #pattern size in pixels
radius=1.2 #reciprocal radius
ediff = pxm.DiffractionGenerator(300., 0.025) #eV and relrod size

We now create 4 seperate patterns, 2 for each crystal, one at 0deg and one at 45deg

In [162]:

thetas = [0,45]

In [163]:
def create_pattern(rot,sample):
    # This functions needs global variables size,radius and ediff (see 2 boxes up)
    transformed = rot.apply_transformation(sample)
    diff_data   = ediff.calculate_ed_data(transformed,radius)
    return diff_data.as_signal(size,0.05,radius).data

In [164]:
data_silicon = []
data_gallium = []
for theta in [0,pi/4]:
    axis, angle = euler2axangle(0,0,theta, 'rzxz') #using axis angle to avoid ambiguity
    rot = RotationTransformation(axis, angle,angle_in_radians=True)
    data_silicon.append(create_pattern(rot,si))
    data_gallium.append(create_pattern(rot,ga))
    
data = [data_silicon] + [data_gallium]
test_data = pxm.ElectronDiffraction(data)
test_data.set_calibration(radius/128)

test_data now contains the 4 patterns we will attempt to match to, we move onto creating a library

In [165]:
rot_list = []
for theta in np.linspace(0,(pi/3),100): #rotations between 0 and 60 deg
    rot_list.append((theta, 0., 0.))    #needs to be a tuple

In [166]:
struc_lib = dict()
struc_lib['si'] = (si, rot_list)
struc_lib['ga'] = (ga,rot_list)

In [167]:
diff_gen = pxm.DiffractionLibraryGenerator(ediff)
library = diff_gen.get_diffraction_library(struc_lib,
                                            calibration=1.2/128,
                                            reciprocal_radius=radius,
                                            representation='euler',
                                            half_shape=(256/2,256/2),
                                            with_direct_beam=False)

which we correlate with the test data

In [168]:
indexer = IndexationGenerator(test_data, library)
phase=["si","ga"] 
match_results = indexer.correlate(n_largest=1,keys=phase)

A Jupyter Widget

This is the "best" result for each setup, column 1 refers to the phase (0 for Si and 1 for Ga), column 2 the z angle, and then 2 and 3 the x and second z angle with 5 being the correlation score

In [169]:
match_results.data

array([[[[ 0.        ,  0.        ,  0.        ,  0.        ,  0.19532511],
         [ 1.        ,  0.        ,  0.        ,  0.        ,  0.00781312]],

        [[ 0.        ,  0.78275373,  0.        ,  0.        ,  0.19232197],
         [ 1.        ,  0.77217597,  0.        ,  0.        ,  0.00802111]]],


       [[[ 0.        ,  0.        ,  0.        ,  0.        ,  0.02606217],
         [ 1.        ,  0.01057775,  0.        ,  0.        ,  0.68666427]],

        [[ 0.        ,  0.77217597,  0.        ,  0.        ,  0.02556211],
         [ 1.        ,  0.78275373,  0.        ,  0.        ,  0.69474166]]]])

Which we can also plot

In [170]:
peaks= match_results.map(peaks_from_best_template,phase=phase,library=library,inplace=False)
mmx,mmy = generate_marker_inputs_from_peaks(peaks)
test_data.plot(cmap='viridis') 
for mx,my in zip(mmx,mmy):
    m = hs.markers.point(x=mx,y=my,color='red',marker='x')
    test_data.add_marker(m,plot_marker=True,permanent=True)

A Jupyter Widget